In [1]:
import pandas as pd
import os

directory = '../../Data/School/0001'
file_name_label = '0001_tei label.txt'

file_label = os.path.join(directory, file_name_label)

In [2]:
file_name_data = '0001_tei.dat'

file_data = os.path.join(directory, file_name_data)
df = pd.read_csv(file_data, sep = '\t', error_bad_lines = False)

In [4]:
with open(file_label, 'r') as f:
    x = f.read().splitlines()

In [5]:
def get_list_of_lists(x):
    # Return each row as a list
    list_of_lists = []
    for line in x:
        list_of_lists.append(line.split('\t'))
    return list_of_lists

In [6]:
list_of_lists = get_list_of_lists(x)

In [7]:
def search_index(keyword, list_of_lists = list_of_lists):
    # Check if the keyword is in the list
    idx_list = []
    for idx, item in enumerate(list_of_lists):
        if keyword in item[0]:
            idx_list.append(idx)
    return idx_list

In [8]:
# Split list_of_lists to variable info list and varuable value list

In [9]:
hensu_joho_idx = search_index('変数情報')[0] # id of 変数情報

In [10]:
hensu_chi_idx = search_index('変数値')[0] # id of 変数値

In [11]:
# split the list of lists
list_of_lists_info = list_of_lists[hensu_joho_idx:hensu_chi_idx] # Contains value informaiotn
list_of_lists_val = list_of_lists[hensu_chi_idx:-1] # Contains value variables

In [12]:
list_of_lists_val_1st_elem = [item[0] for item in list_of_lists_val] # First elements in val list

In [13]:
_list_of_vars_val  = [s for s in list_of_lists_val_1st_elem if s] # list of variables in the values list

In [14]:
list_of_vars_val = _list_of_vars_val[2:]

In [15]:
def return_next_var(var, list_of_vars):
    # Use list of variables, taken from var_info, return the next variable
    current_idx = list_of_vars.index(var)
    next_var = list_of_vars[current_idx + 1]
    return next_var

In [16]:
def get_dict(var, 
             list_of_lists = list_of_lists_val,
             list_of_lists_1st_elem = list_of_lists_val_1st_elem, 
             list_of_vars = list_of_vars_val):
    # Return dictionary for var
    
    dict = {}    
    idx = list_of_lists_1st_elem.index(var)    
    
    if var != list_of_vars[-1]:
        # Get the next variable using the list of variables in the values list
        next_var = return_next_var(var, list_of_vars)
    
        # Get the index for the list of lists of values        
        next_idx = list_of_lists_1st_elem.index(next_var)
        sublist = list_of_lists[idx:next_idx]
        
    else:
        sublist = list_of_lists[idx:]
    # Slice the list of lists
        
    for item in sublist:
        new_dict = {item[1]:item[2]}
        dict.update(new_dict)
    return dict

In [17]:
def replace_val(var, df = df):
    dictionary = get_dict(var)
    df[var] = df[var].astype(str)
    df[var] = df[var].map(dictionary)

In [18]:
for var in list_of_vars_val:
    replace_val(var)

In [20]:
# Can be used to replace column names
variables_list = [(item[0], item[2]) for item in list_of_lists_info[3:-1]]

In [21]:
variables_dict = dict(variables_list)

In [23]:
df.rename(columns = variables_dict, inplace=True)

In [24]:
df

,gakkou,性別,跡取りか,保護者との続柄,保護者の職業,学業成績,学科,将来の希望
0,101,男子,無回答,長男・長女,俸給生活者,上,NaN,就職
1,101,男子,無回答,長男・長女,俸給生活者,上,NaN,進学
2,101,男子,無回答,四男・四女以上,独立工業者,中,職業科,就職
3,101,男子,無回答,長男・長女,工員,上,職業科,就職
4,101,男子,無回答,次男・次女,無職,上,NaN,進学
...,...,...,...,...,...,...,...,...
16559,1312,女子,無回答,次男・次女,農業,上,NaN,就職
16560,1312,女子,無回答,三男・三女,農業,上,職業科,就職
16561,1312,女子,無回答,四男・四女以上,農業,上,NaN,進学
16562,1312,女子,無回答,四男・四女以上,農業,上,職業科,就職
